# Import dependencies and libraries

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

# Twitter API authentication

In [2]:
# keys and tokens from the Twitter Dev Console 
consumer_key = '4Skk3kFGkHb0ryTGLvRNcrN3M'
consumer_secret = 'O37XRtd5HsPy05r2H6eJnx2nWWIcVrbnVJXWDwPmPcCacmrZMx'
access_token = '1214729489730678784-vfBALf5kgI1YsORCj8fiktzaqPwUKW'
access_token_secret = '3BoSyzrQW6UibJNU493N64rF5BZRdEIFjatAqcS2yNicG'

# Twitter API authentication
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Custom search criteria

In [3]:
# Create a custom search term and define the number of tweets
search_term = "#AAPL -filter:retweets"

# Functions to return a dataframe -  a list of cleaned up tweets for custom search criteria , its score and polarity 

As we agreed to just pull information based on user input for 1 ticker at a time, functions are created below to pull for a particular ticker

In [6]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

def tweets():
    tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2019-11-01').items(1000)

    # Remove URLs
    tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]
    
    # Create textblob objects of the tweets
    sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]
    sentiment_objects[0].polarity, sentiment_objects[0]
    
    # Create list of polarity valuesx and tweet text
    sentiment_values = [[str(tweet),tweet.sentiment.polarity,'Positive' if tweet.sentiment.polarity > 0 else 'Negative' if tweet.sentiment.polarity < 0 else 'Neutral'] for tweet in sentiment_objects]
    sentiment_values[0]
    
    # Create dataframe containing the polarity value and tweet text
    sentiment_df = pd.DataFrame(sentiment_values, columns=[ "Tweet","Score","Polarity"])
    return sentiment_df
    
    
    

In [7]:
AAPL_tweets=tweets()
AAPL_tweets

,Tweet,Score,Polarity
0,Last time AAPL was this far above its 200 day ...,0.033333,Positive
1,AAPL chart via,0.000000,Neutral
2,CNBC SquawkAlley jimcramer carlquintanilla jon...,0.800000,Positive
3,realDonaldTrump any tweet today that fix thisF...,0.000000,Neutral
4,AAPL AAPL another high just doesnt stop in calls,0.160000,Positive
5,AAPLIn last 9 sessions including todays market...,0.000000,Neutral
6,AAPL A double for the year BitcoinSV BSV a dou...,0.266667,Positive
7,Apple AAPL Stock will not hold these new highs...,0.099432,Positive
8,AAPL chart via AAPL is reaching the extension ...,0.000000,Neutral
9,AAPL chart via APPL is reaching the extension ...,0.000000,Neutral
